Imports

In [2]:
import cv2
import numpy as np
import numpy as np
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
import os
from sklearn.model_selection import train_test_split

Load Dataset

In [ ]:
def load_data(images_folder, segmented_folder):
    X = []  # Image data (character images)
    y = []  # Corresponding labels (characters)

    # Loop through each image in the images folder
    for license_plate_image in os.listdir(images_folder):
        if license_plate_image.endswith('.png'):
            license_plate_name = license_plate_image.split('.')[0]  
            # Path to the folder containing segmented characters for this license plate
            segmented_path = os.path.join(segmented_folder, license_plate_name)

            # Loop through the segmented characters in the corresponding folder
            for char_img_name in os.listdir(segmented_path):
                if char_img_name.endswith('.png'):
                    # Load character image
                    char_img_path = os.path.join(segmented_path, char_img_name)
                    char_image = cv2.imread(char_img_path, cv2.IMREAD_GRAYSCALE)
                    
                    # Get the corresponding label (the first character in the name of the license plate image)
                    label = license_plate_name[int(char_img_name.split('.')[0])]  

                    # Append the character image and label to the lists
                    X.append(char_image)
                    y.append(label)

    # Convert lists to numpy arrays for easier manipulation
    X = np.array(X)
    y = np.array(y)

    return X, y

# Example usage:
segmented_folder = "C:/Users/Cipleu/Documents/IULIA/SCOALA/facultate/Year 4 Semester 1/PRS/Lab/Project/charactersResulted"  
images_folder = "C:/Users/Cipleu/Documents/IULIA/SCOALA/facultate/Year 4 Semester 1/PRS/Lab/Project/dataset/imagesForTesting/plates"  

X, y = load_data(images_folder, segmented_folder)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

IndexError: string index out of range

Histogram of Oriented Gradients (HOG)

In [8]:
import cv2
import numpy as np

class HOGDescriptor:
    def __init__(self, cell_size=(8, 8), block_size=(2, 2), nbins=9):
        self.cell_size = cell_size
        self.block_size = block_size
        self.nbins = nbins

    def compute_gradients(self, image):
        # Compute gradients along x and y directions
        gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        
        # Compute magnitude and angle
        magnitude = np.sqrt(gx**2 + gy**2)
        angle = np.arctan2(gy, gx) * (180 / np.pi) % 180  # Angle in degrees between 0 and 180
        
        return magnitude, angle

    def compute_histograms(self, magnitude, angle):
        # Divide the image into cells and compute histograms
        cell_histograms = []
        for i in range(0, magnitude.shape[0], self.cell_size[0]):
            for j in range(0, magnitude.shape[1], self.cell_size[1]):
                cell_magnitude = magnitude[i:i + self.cell_size[0], j:j + self.cell_size[1]]
                cell_angle = angle[i:i + self.cell_size[0], j:j + self.cell_size[1]]
                
                # Compute histogram for each cell
                hist = np.zeros(self.nbins)
                for m, a in zip(cell_magnitude.flatten(), cell_angle.flatten()):
                    bin_idx = int(a // (180 / self.nbins))  # Bin index
                    hist[bin_idx] += m  # Add magnitude to corresponding bin
                
                cell_histograms.append(hist)
        return np.array(cell_histograms)

    def compute(self, image):
        # Resize image
        image_resized = cv2.resize(image, (128, 64))
        print(f"Resized Image Shape: {image_resized.shape}")
        
        # Compute gradients and histogram of gradients
        magnitude, angle = self.compute_gradients(image_resized)
        print(f"Gradient Magnitude Shape: {magnitude.shape}")
        print(f"Gradient Angle Shape: {angle.shape}")
        
        # Compute histograms for cells
        cell_histograms = self.compute_histograms(magnitude, angle)
        print(f"Number of Cells: {len(cell_histograms)}")
        
        # Group cells into blocks and normalize histograms
        block_histograms = []
        for i in range(0, len(cell_histograms) - (self.block_size[0] - 1) * 8, self.block_size[1]):
            block = cell_histograms[i:i + self.block_size[0] * self.block_size[1]]
            block_histograms.extend(block)
        
        # L2-norm normalization
        block_histograms = np.array(block_histograms)
        block_histograms /= np.sqrt(np.sum(block_histograms**2) + 1e-6)
        
        print(f"Block Histograms Length: {len(block_histograms)}")
        return block_histograms

# Example usage:
image = cv2.imread('C:/Users/Cipleu/Documents/IULIA/SCOALA/facultate/Year 4 Semester 1/PRS/Lab/Project/charactersResulted/YWORRY/0.png', cv2.IMREAD_GRAYSCALE)  # Load grayscale image
hog = HOGDescriptor()
features = hog.compute(image)

# Print some of the computed features (optional)
print(f"Computed HOG Features: {features[:10]}")  # Print the first 10 HOG features


Resized Image Shape: (64, 128)
Gradient Magnitude Shape: (64, 128)
Gradient Angle Shape: (64, 128)
Number of Cells: 128
Block Histograms Length: 240
Computed HOG Features: [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.96251843e-01 2.41667948e-02 3.70202273e-03 1.46697018e-03
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.41396242e-01 0.00000000e+00 7.23251167e-05 2.15038031e-04
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.41396242e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.39342597e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.42561894e-01 3.82588024e-03 7.19970459e-04 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.41396242e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.39342597e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.42561894e-01 3.82588024e-03 7.19970459e-04 0.0000000

Naive Bayesian Classifier

In [13]:
import numpy as np
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = defaultdict(float)  # Class probabilities
        self.feature_probs = defaultdict(lambda: defaultdict(float))  # Feature probabilities given class

    def train(self, X_train, y_train):
        """
        Train Naive Bayes classifier.
        
        :param X_train: List of feature vectors
        :param y_train: List of class labels
        """
        n_samples = len(X_train)
        class_counts = defaultdict(int)
        
        # Count occurrences of each class
        for label in y_train:
            class_counts[label] += 1
        print("Class counts:", class_counts)
        
        # Calculate class probabilities
        for label, count in class_counts.items():
            self.class_probs[label] = count / n_samples
        print("Class probabilities:", self.class_probs)
        
        # Calculate feature probabilities given each class
        feature_counts = defaultdict(lambda: defaultdict(int))
        for i in range(n_samples):
            label = y_train[i]
            features = X_train[i]
            for feature in features:
                feature_counts[label][feature] += 1

        for label, features in feature_counts.items():
            total_features = sum(features.values())
            print(f"Total features for class {label}: {total_features}")
            for feature, count in features.items():
                self.feature_probs[label][feature] = count / total_features
                print(f"Feature probabilities for class {label}, feature {feature}: {self.feature_probs[label][feature]}")

    def predict(self, X_test):
        """
        Predict class labels for the given test data.
        
        :param X_test: List of feature vectors to classify
        :return: List of predicted class labels
        """
        predictions = []
        for features in X_test:
            class_scores = {}
            print(f"\nPredicting for features: {features}")
            for label, class_prob in self.class_probs.items():
                score = np.log(class_prob)
                print(f"Class {label} initial score (log prior): {score}")
                for feature in features:
                    feature_prob = self.feature_probs[label].get(feature, 1e-5)  # Apply Laplace smoothing
                    score += np.log(feature_prob)
                    print(f"Feature {feature} score for class {label}: {np.log(feature_prob)}")
                class_scores[label] = score
                print(f"Total score for class {label}: {score}")
            predicted_class = max(class_scores, key=class_scores.get)
            print(f"Predicted class: {predicted_class}")
            predictions.append(predicted_class)
        return predictions


Voting Mechanism

In [16]:
class VotingClassifier:
    def __init__(self, classifiers, weights=None):
        self.classifiers = classifiers
        self.weights = weights if weights else [1] * len(classifiers)
    
    def fit(self, X_train, y_train):
        for clf in self.classifiers:
            clf.train(X_train, y_train)  # Train each classifier

    def predict(self, X_test):
        # Get predictions from all classifiers
        all_preds = []
        for clf in self.classifiers:
            all_preds.append(clf.predict(X_test))
        
        # Perform weighted voting
        weighted_preds = np.zeros(len(X_test))
        for i, preds in enumerate(zip(*all_preds)):
            scores = np.zeros(len(set(preds)))  # Assuming class labels are integers
            for j, pred in enumerate(preds):
                scores[pred] += self.weights[i]
            weighted_preds[i] = np.argmax(scores)  # Choose class with max score
        return weighted_preds

# Example usage:
classifiers = [
    ('naive_bayes', NaiveBayesClassifier()),
    ('random_forest', RandomForestClassifier(n_estimators=100))
]

voting_clf = VotingClassifier(estimators=classifiers, voting='hard')  # Use 'hard' voting for majority class
voting_clf.fit(X_train, y_train)

# Predict using the voting classifier
predictions = voting_clf.predict(X_test)


TypeError: VotingClassifier.__init__() got an unexpected keyword argument 'estimators'

Main